# TD twitter bot classification

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from pycaret.classification import (
    setup, compare_models, plot_model, blend_models,
    tune_model, save_model, load_model, finalize_model)
from pycaret.regression import interpret_model

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'DejaVu Sans'

# I. Chargement des données
Charger le csv `twitter_human_bots_dataset.csv` dans un dataframe que vous nommerez `df`

In [ ]:
df = pd.read_csv(# À completer)
print(f"Dataset shape {df.shape}")
df.head(5)

# II. Statistiques descriptives

**Afficher le nombre de bots et le nombre d'humains (voir la colonne `account_type`)**

In [ ]:
# À completer

**Afin de mettre en lien la "target" (`account_type`) et la variable `verified`, afficher un bar plot en affichant une couleur différente en fonction de la colonne `verified`**

Indice : `account_type` en x, le nombre d'occurence par "target" en y

In [ ]:
df_plot = df.groupby(["account_type", "verified"])["account_type"].count().unstack("verified")
df_plot.plot(kind='bar', stacked=True)

### Facets Dive

Dive est un outil permettant d'explorer de manière interactive un grand nombre de points de données à la fois. Il fournit une interface interactive pour explorer la relation entre les points de données à travers toutes les différentes caractéristiques d'un ensemble de données. Chaque élément individuel de la visualisation représente un point de données. Positionnez les éléments en les "facettant" ou en les classant dans plusieurs dimensions en fonction des valeurs de leurs caractéristiques.

Demo : https://pair-code.github.io/facets/quickdraw.html

In [ ]:
df_sample = df.sample(10000)

In [ ]:
# Display the Dive visualization for the training data.
from IPython.core.display import display, HTML

jsonstr = df_sample.to_json(orient='records')
HTML_TEMPLATE = """
        <script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html">
        <facets-dive id="elem" height="600"></facets-dive>
        <script>
          var data = {jsonstr};
          document.querySelector("#elem").data = data;
        </script>"""
html = HTML_TEMPLATE.format(jsonstr=jsonstr)
display(HTML(html))

# III. Classifier de bot

## a. Features engineering
Un modèle de Machine Learning ne peut prendre en entrée que des entiers ou des réels. L'objectif de cette partie est de transformer intelligement les données brutes en variables (features en anglais) intelligibles pour notre modele.

In [ ]:
df.iloc[0]

**Créer une variable `X` qui est une copie de `df`. C'est maintenant sur cette variable `X` que nous allons travailler**

Indice : utiliser `.copy()`

In [ ]:
X = # À completer

**Convertisser les variables `"default_profile", "default_profile_image", "geo_enabled", "verified"` en entier**

Indice : utiliser `.astype(int)`

In [ ]:
# Preprocess boolean columns
for col in ["default_profile", "default_profile_image", "geo_enabled", "verified"]:
    X[col] = X[col]# À completer
X.head()

**Créér une nouvelle colonne que vous nommerez `popularity` en appelant la fonction `compute_popularity_metric` qui ajoute un score de "popularité" en fonction du nombre d'amis et du nombre de followers**

In [ ]:
def popularity_metric(friends_count: int, followers_count: int):
    return np.round(np.log(1+friends_count) * np.log(1+followers_count), 3)

def compute_popularity_metric(row):
    return popularity_metric(friends_count=row["friends_count"],
                             followers_count=row["followers_count"])

X["popularity"] = X.apply(# À completer)

In [ ]:
X[['popularity']].hist()

In [ ]:
df_plot = X.copy()
df_plot['popularity_bins'] = pd.cut(
    df_plot['popularity']/df_plot['popularity'].max()*100,
    bins=list(range(0, 110, 20)))
# plot
(df_plot.groupby(["account_type", 'popularity_bins'])["account_type"]
    .count()
    .unstack('popularity_bins')
    .plot(kind='bar', stacked=True))

**Convertir la colonne target en catégorie**

Indices :
- vous devez caster la colonne `account_type` en `category` avec `.astype('category')`
- écraser ensuite les valeurs de la colonne avec `.cat.codes` pour récupérer un entier

In [ ]:
X["account_type"] = X["account_type"]# À completer
X["account_type"] = X["account_type"]# À completer
X["account_type"]

## b. Features selection

**Ne garder maintenant que les valeurs numériques**

Indice : utiliser `_get_numeric_data()`

In [ ]:
X = # À completer

**Supprimer la colonne `id`, elle ne porte aucune information pour le modele de classification**

Indice : utiliser `drop(..., axis=1,inplace=True)`

In [ ]:
# À completer

**Calculer la matrice de correlations des variables**

Indice : utiliser `.corr()`

In [ ]:
corr = # À completer

**Afficher cette matrice de correlations**

In [ ]:
sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,
            annot=True, fmt='.1g', cmap='coolwarm')

## c. AutoML

**Séparer votre dataset `X` en deux sets de données via la méthode `train_test_split`. Utiliser `0.2` (=20% du dataset) pour la proportion du `test_size`**

In [ ]:
dataset_train, dataset_unseen = train_test_split(# À completer)

**Il est maintenant temps de préparer la classification avec l'appel à la fonction `setup` de PyCaret !**

In [ ]:
%%time
setup(# À completer)

**Passez au crible les modèles de PyCaret et comparez leurs performances.**

Quel est le modèle avec la meilleure exactitude ?

Indice : utiliser `compare_models` de PyCaret

In [ ]:
%%time
best_model = # À completer

**Affichez la matrice de confusion sur les données de test.**

Que pouvez-vous en dire ?

Indice : utiliser `plot_model` de PyCaret

In [ ]:
# À completer

## d. Pour aller plus loin

Pour améliorer les performances, il est possible de faire varier les hyper-paramètres du modèle.

Cela peut se faire à l'aide d'un simple appel à la fonction `tune_model`, [décrite ici](https://pycaret.org/tune-model/).

<b>Reprenez le meilleur modèle et optimisez ses hyper-paramètres.</b> Constatez-vous une amélioration des performances ?

In [ ]:
%%time
best_model_tuned = # À completer

Une autre piste d'amélioration réside dans la **combinaison de plusieurs modèles**, et le **blending** de leurs prédictions via un vote majoritaire.

Cela peut se faire à l'aide d'un simple appel à la fonction `blend_models`, [décrite ici](https://pycaret.org/blend_models/).

Reprenez les 5 meilleur modèles et effectuez un blending. Constatez-vous une amélioration des performances en test ?

In [ ]:
%%time
top_5_models = # À completer

blended_model = # À completer

<b>Ici aussi, on note une stagnation des performances, avec presque la même matrice de confusion.</b>

Pour conclure ce TP, on se propose de fournir une explication <i>a posteriori</i> du fonctionnement du classifieur.

Pour obtenir le graphique d'importance des variables explicatives, appelez la fonction `plot_model` avec comme argument le modèle de votre choix et avec l'argument `plot="feature"`.

Comment interprétez-vous ces valeurs ?

In [ ]:
# À completer

**Afficher la courbe ROC**

Indice : utiliser `plot_model` avec  `plot='auc'`

In [ ]:
# À completer

**Afficher la courbe Précision-Recall**

Indice : utiliser `plot_model` avec  `plot='pr'`

In [ ]:
# À completer

**Afficher l'explicabilité**

Indice : utiliser `interpret_model`

In [ ]:
# À completer

**Entrainer le modele final**

Indice : utiliser `finalize_model` en réutilisant le best model tuned

In [ ]:
final_model = # À completer

**Sauvegarder le modele final**

Indice : utiliser `save_model`

In [ ]:
# À completer

**Recharger le modele final**

Indice : utiliser `load_model`

In [ ]:
# À completer